In [3]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import Image, ImageTk
import numpy as np
import cv2
import dlib

In [20]:
# (對68個點要變動的程式碼，每對imgS做變動，都要呼叫Renew()函式更新)

def OpenCV_small():  # opencv 等比縮小圖片
    global imgS    
    nr,nc = imgS.shape[:2]
    if nr >= 800 or nc >= 800:
        if nr>= nc:
            scale = 800/nr 
        else:
            scale = 800/nc
        nr2 = int(nr * scale)
        nc2 = int(nc * scale)
        imgS = cv2.resize(imgS,(nc2,nr2),interpolation = cv2.INTER_LINEAR)

def open_file():
    global imgS,img_show,panel
    filename=filedialog.askopenfilename()  #獲取文件全路徑
    imgS = cv2.imread(filename) # 用opencv的方法    
    OpenCV_small() # opencv 等比縮小圖片
    
    # Label改成Canvas，不要用Label因為會有覆寫問題
    panel = tk.Canvas(block1,width=650,height=800,bg='#FFFFFF') # 設定長寬   
    imgS = rection(imgS)  # imgS是rection後的opencv圖    
    Renew() # 更新

def Renew():
    global imgS,img_show,panel
    # 把img_show變成PIL格式圖
    img_show = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    img_show = Image.fromarray(img_show)
    img_show = ImageTk.PhotoImage(image = img_show)
    # 要更新
    panel.delete("all") 
    panel.create_image(0,0,image=img_show,anchor=NW)
    panel.place(x=0,y=0)

def rection(img): # OpenCV 測68個點
    #dlib預測器
    detector = dlib.get_frontal_face_detector()    #使用dlib庫提供的人臉提取器
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')   #構建特徵提取器
    # 取灰度
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 人臉數rects
    rects = detector(img_gray, 0)
    for i in range(len(rects)):
        landmarks = np.matrix([[p.x, p.y] for p in predictor(img,rects[i]).parts()])  #人臉關鍵點識別
        for idx, point in enumerate(landmarks):        #enumerate函式遍歷序列中的元素及它們的下標
            # 68點的座標
            pos = (point[0, 0], point[0, 1])
            print(idx,pos)

            # 利用cv2.circle給每個特徵點畫一個圈，共68個
            cv2.circle(img, pos, 4, color=(0, 255, 0),thickness = -1)
            # 利用cv2.putText輸出1-68
            #font = cv2.FONT_HERSHEY_SIMPLEX
            #各引數依次是：圖片，新增的文字，座標，字型，字型大小，顏色，字型粗細
            #cv2.putText(img, str(idx+1), pos, font, 0.8, (0, 0, 255), 1,cv2.LINE_AA)
    return img

In [21]:
global imgS,img_show,panel
   
window = tk.Tk()
window.title('人臉五官微調系統')
window.geometry('960x800')

div_size = 300
align_mode = 'nswe'
pad0 = 10
pad = 5

#GUI整體布局
blocktop = tk.Frame(window, width=650, height=30)
blocktop2 = tk.Frame(window, width=div_size, height=30)
block1 = tk.Frame(window, width=650, height=1000,bg='#FFFFFF')
block2 = tk.Frame(window, width=div_size, height=250)
block3 = tk.Frame(window, width=div_size, height=250)
block4 = tk.Frame(window, width=div_size, height=250)
block5 = tk.Frame(window, width=div_size, height=250)

blocktop.grid(column=0, row=0, sticky=align_mode)
blocktop2.grid(column=1, row=0, sticky=align_mode)
block1.grid(column=0, row=1, rowspan=5, sticky=align_mode) 
block2.grid(column=1, row=2, sticky=align_mode)
block3.grid(column=1, row=3, sticky=align_mode)
block4.grid(column=1, row=4, sticky=align_mode)
block5.grid(column=1, row=5, sticky=align_mode)


#拉霸區塊布局
facelabel = tk.Label(block5,text="臉型",font=('新細明體', 12),padx=pad, pady=pad,fg='#007799')
font = ('Courier New', 20, 'bold')
facescale1 = tk.Scale(
    block5, label='瘦臉', from_=0, to=100, orient="horizontal",tickinterval=10,length=280)

facelabel.grid(row=0, column=0)
facescale1.grid(row=1, column=0)



eyelabel = tk.Label(block2,text="眼睛",font=('新細明體', 12),padx=pad, pady=pad,fg='#007799')
font = ('Courier New', 20, 'bold')
eyescale1 = tk.Scale(
    block2, label='大小', from_=0, to=100, orient="horizontal",tickinterval=10,length=280)
eyescale2 = tk.Scale(
    block2, label='旋轉', from_=0, to=100, orient="horizontal",tickinterval=10,length=280)
eyescale3 = tk.Scale(
    block2, label='眼距', from_=0, to=100, orient="horizontal",tickinterval=10,length=280)

eyelabel.grid(row=0, column=0)
eyescale1.grid(row=1, column=0)
eyescale2.grid(row=2, column=0)
eyescale3.grid(row=3, column=0)

noselabel = tk.Label(block3,text="鼻子",font=('新細明體', 12),padx=pad, pady=pad,fg='#007799')
nosescale1 = tk.Scale(
    block3, label='大小', from_=0, to=100, orient="horizontal",tickinterval=10,length=280)
nosescale2 = tk.Scale(
    block3, label='鼻翼', from_=0, to=100, orient="horizontal",tickinterval=10,length=280)

noselabel.grid(row=0, column=0)
nosescale1.grid(row=1, column=0)
nosescale2.grid(row=2, column=0)

mouselabel = tk.Label(block4,text="嘴巴",font=('新細明體', 12),padx=pad, pady=pad,fg='#007799')
mousescale1 = tk.Scale(
    block4, label='大小', from_=0, to=100, orient="horizontal",tickinterval=10,length=280)
mousescale2 = tk.Scale(
    block4, label='薄厚', from_=0, to=100, orient="horizontal",tickinterval=10,length=280)

mouselabel.grid(row=0, column=0)
mousescale1.grid(row=1, column=0)
mousescale2.grid(row=2, column=0)


#開檔&存檔
import_btn = tk.Button(blocktop, text='開啟檔案', bg='#BBFFEE', fg='black',height = 1, 
          width = 20, command=open_file)
save_btn = tk.Button(blocktop, text="儲存檔案", bg='#BBFFEE', fg='black',height = 1, 
          width = 20)
import_btn.grid(column=0, row=0, padx=pad0, pady=pad0, sticky=align_mode)
save_btn.grid(column=1, row=0, padx=pad0, pady=pad0, sticky=align_mode)

#顯示照片
mainImage=tk.Label(block1,height = 1, width = 95, image=None,bg='#FFFFFF') 
mainImage.grid(row=0, column=0, sticky=align_mode)


window.mainloop()

0 (210, 305)
1 (214, 342)
2 (222, 379)
3 (230, 415)
4 (243, 452)
5 (263, 484)
6 (290, 512)
7 (321, 533)
8 (357, 538)
9 (391, 531)
10 (420, 508)
11 (442, 477)
12 (458, 444)
13 (467, 408)
14 (474, 372)
15 (479, 336)
16 (480, 300)
17 (237, 284)
18 (257, 270)
19 (283, 270)
20 (308, 276)
21 (332, 286)
22 (367, 286)
23 (390, 274)
24 (413, 268)
25 (439, 268)
26 (459, 282)
27 (350, 313)
28 (351, 338)
29 (351, 364)
30 (352, 391)
31 (329, 408)
32 (341, 412)
33 (352, 415)
34 (364, 411)
35 (375, 407)
36 (265, 312)
37 (280, 303)
38 (299, 305)
39 (313, 318)
40 (297, 321)
41 (278, 320)
42 (386, 318)
43 (401, 304)
44 (419, 302)
45 (434, 311)
46 (421, 319)
47 (403, 320)
48 (303, 452)
49 (323, 446)
50 (340, 441)
51 (353, 444)
52 (366, 440)
53 (382, 445)
54 (400, 450)
55 (383, 467)
56 (367, 476)
57 (353, 478)
58 (339, 477)
59 (321, 468)
60 (311, 455)
61 (340, 456)
62 (353, 457)
63 (366, 456)
64 (393, 452)
65 (366, 455)
66 (353, 458)
67 (340, 456)
